In [30]:
from datasets import load_from_disk, Dataset
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from tqdm import tqdm, trange
import torch
import torch.nn.functional as F
#from retrieval_model import BertEncoder
from transformers import AdamW, TrainingArguments, get_linear_schedule_with_warmup
from torch import nn
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset, SequentialSampler)
from tqdm import tqdm, trange
import pickle
import os, json

In [31]:
train_dataset = load_from_disk("data/new_train_dataset/train")
valid_dataset = load_from_disk("data/new_train_dataset/validation")
wiki_dataset = pd.read_json('data/preprocess_wiki.json',orient='index') # wiki context
data_path = "data/"
context_path = "preprocess_wiki.json"
with open(os.path.join(data_path, context_path), "r", encoding="utf-8") as f:
    wiki = json.load(f)

# wiki_dataset = dict(
#     dict.fromkeys([v["text"] for v in wiki.values()])
# ) 


test_dataset = load_from_disk('data/test_dataset/validation')


In [32]:
valid_corpus = list(set(valid_dataset['context']))
train_corpus = list(set(train_dataset['context']))
wiki_corpus = list(set(wiki_dataset))
query = valid_dataset['question']
test_query = test_dataset['question']

In [33]:
from elasticsearch import Elasticsearch, helpers
import pprint  

In [34]:
INDEX_NAME = "wiki_index"

INDEX_SETTINGS = {
  "settings" : {
    "index":{
      "analysis":{
        "analyzer":{
          "korean":{
            "type":"custom",
            "tokenizer":"nori_tokenizer",
            "filter": [ "shingle" ],

          }
        }
      }
    }
  },
  "mappings": {

      "properties" : {
        "content" : {
          "type" : "text",
          "analyzer": "korean",
          "search_analyzer": "korean"
        },
        "title" : {
          "type" : "text",
          "analyzer": "korean",
          "search_analyzer": "korean"
        }
      }

  }
}

In [35]:
docs = [
    {
        '_index' : INDEX_NAME,
        '_id' : wiki_dataset.iloc[i]['document_id'],
        'title' : wiki_dataset.iloc[i]['title'],
        'content' : wiki_dataset.iloc[i]['text']
    }
    for i in range(wiki_dataset.shape[0])
]

In [36]:
try:
    es.transport.close()
except:
    pass
es = Elasticsearch()

In [37]:
es.info()

<ipython-input-37-f69669341f07>:1: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  es.info()


{'name': 'deeeb6a2da8a',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'ITGy6aY8T8qlIhgwnVJprQ',
 'version': {'number': '7.15.1',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '83c34f456ae29d60e94d886e455e6a3409bba9ed',
  'build_date': '2021-10-07T21:56:19.031608185Z',
  'build_snapshot': False,
  'lucene_version': '8.9.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [38]:
if es.indices.exists(INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)

<ipython-input-38-b31c6028f9a8>:1: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  if es.indices.exists(INDEX_NAME):
<ipython-input-38-b31c6028f9a8>:2: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  es.indices.delete(index=INDEX_NAME)
<ipython-input-38-b31c6028f9a8>:3: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  es.

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'wiki_index'}

In [39]:
import time
start_time = time.time()
try:
    response = helpers.bulk(es, docs)
    print ("\nRESPONSE:", response)
except Exception as e:
    print("\nERROR:", e)
    
end_time = time.time()

print(end_time-start_time)

/opt/conda/lib/python3.8/site-packages/elasticsearch/helpers/actions.py:122: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  resp = client.bulk("\n".join(bulk_actions) + "\n", *args, **kwargs)



RESPONSE: (60613, [])
94.50452733039856


In [40]:
questions = valid_dataset['question']

In [41]:
answers = valid_dataset['answers']

In [43]:
def get_hit_rate(questions, answers, size):
    error_questions = []
    hit_cnt = 0
    for question, answer in zip(questions, answers):
        # print(question)
        # print(answer)
        # break
        try:
            res = es.search(index=INDEX_NAME, q=question, size=size)
        except Exception as e:
            error_questions.append([e, question])
        for hit in res['hits']['hits']:
            if answer['text'][0] in (hit['_source']['title']+hit['_source']['content']):
                hit_cnt += 1
                break
    print('size : '+str(size)+'\nhit rate', hit_cnt/len(questions))
    return error_questions
# for size in range(5, 31):
for size in range(0, 10):
    get_hit_rate(questions, answers, size)


<ipython-input-43-fba2dbb27aa2>:9: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  res = es.search(index=INDEX_NAME, q=question, size=size)


size : 0
hit rate 0.0
size : 1
hit rate 0.7458333333333333
size : 2
hit rate 0.7875
size : 3
hit rate 0.8333333333333334
size : 4
hit rate 0.8583333333333333
size : 5
hit rate 0.8791666666666667
size : 6
hit rate 0.8875
size : 7
hit rate 0.9041666666666667
size : 8
hit rate 0.9166666666666666
size : 9
hit rate 0.9208333333333333


In [168]:
len(test_query)

600

In [169]:
test_query[0]

"유령'은 어느 행성에서 지구로 왔는가?"

In [170]:
test_dataset[0]

{'id': 'mrc-1-000653', 'question': "유령'은 어느 행성에서 지구로 왔는가?"}

In [173]:
def search_query(questions, size):
    error_queries = []
    results = []
    for question in questions:
        try:
            res = es.search(index=INDEX_NAME, q=question, size=size)
            results.append(res)
        except:
            error_queries.append(question)
    return results, error_queries

res, err = search_query(["안녕"], 2)
res, err

<ipython-input-173-335142544b8c>:6: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  res = es.search(index=INDEX_NAME, q=question, size=size)


([{'took': 0,
   'timed_out': False,
   '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
   'hits': {'total': {'value': 136, 'relation': 'eq'},
    'max_score': 12.720648,
    'hits': [{'_index': 'wiki_index',
      '_type': '_doc',
      '_id': '28707',
      '_score': 12.720648,
      '_source': {'title': '안녕, 절망선생 (애니메이션)',
       'content': '대한민국에서는 정식 발매되지 않았다. * 안녕, 절망선생 :애니메이션 1기. 2007년 7월부터 9월까지의 방송을 4장에 나눠 발매했으며, 특장판과 일반판으로 나뉘여져있다. * 절망소녀찬집 :애니메이션판 다이제스트 DVD. 애니메이션 1기보다 더욱 원작에 충실하며, 히토 나미의 이야기를 중간중간에 끼워넣었다. 2008년 1월 1일 발매. * 속 안녕, 절망선생 :애니메이션 2기. 2008년 1월부터 3월까지의 방송을 4장에 나눠 발매했으며, 특장판과 일반판으로 나뉘여져있다. * 속 절망소녀찬집 :애니메이션판 다이제스트 DVD. 절망소녀찬집의 후속작이며, 애니메이션 2기의 내용보다 더욱더 원작에 충실한 내용들로 구성되어 있다. 2008년 8월 27일 발매. 특전영상으로 절망문학집, 에피소드 『은폐졸』의 오리지널 성우진 버전을 수록하였다.(원작에는 성우진 캐스팅을 바꿔서 녹음하였다.) * 옥 안녕, 절망선생 :안녕, 절망선생의 오리지널 애니메이션 DVD. 3장으로 구성되어있으며, 각각 2008년 10월 17일, 2008년 12월 10일, 2009년 2월 17일에 발매되었다. * 참 안녕, 절망선생 :애니메이션 3기. 2009년 7월부터 9월까지의 방송을 4장에 나눠 발매했으며, 특장판과 일반판으로 나뉘여져있다. * 참

In [172]:
res, err = search_query(test_query, 20)

<ipython-input-171-73c60fbcc4a2>:6: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  res = es.search(index=INDEX_NAME, q=question, size=size)


In [51]:
res[0]['hits']['hits'][0]

{'_index': 'wiki_index',
 '_type': '_doc',
 '_id': '43280',
 '_score': 20.487246,
 '_source': {'title': '거대 행성',
  'content': '목성의 대기에서 보이는 줄무늬는 적도와 평행하면서 행성을 둘러싸는 대(zone)와 띠(belt)라고 불리는 물질의 반대 순환류에 의한 것이다. 대는 밝은 줄무늬로, 대기에서 상대적으로 고도가 높은 곳에 있다. 이들은 내부의 상승 기류를 가지고 있는 고기압 영역이다. 띠는 어두운 줄무늬로, 대기에서 상대적으로 고도가 낮은 곳에 있으며, 내부의 하강 기류를 가진다. 이들은 저기압 영역이다. 이러한 구조는 지구 대기의 고기압 및 저기압 세포와 어느정도 유사하나, 국지 작은 기압 세포와 상반되는 행성 전체를 둘러싸는 위도 줄무늬로서 매우 다른 구조를 가지고 있다. 이는 행성의 빠른 자전과 근본적인 대칭으로 인한 결과로 보인다. 행성에는 국지적인 가열을 일으키는 바다나 육지가 없으며 자전 속도는 지구보다 훨씬 빠르다. 행성에는 서로 다른 크기와 색상을 갖는 점과 같은 작은 구조들이 있다. 목성에서, 그러한 특색 중에서 가장 유명한 것은 대적점으로, 적어도 300년 동안 존재해 왔다. 이러한 구조의 실체는 거대한 폭풍이다. 그러한 점 중에 일부는 적란운이기도 하다.\x7f'}}

In [88]:
from datasets import (load_from_disk, load_dataset, Features, 
                        Value, Sequence, DatasetDict, Dataset)

k = 20

test_datas = []
for i in range(len(res)):
    contents = []
    passages = []
    for j in range(k):
        content = {
            'docs_id': res[i]['hits']['hits'][j]['_id'],
            'score': res[i]['hits']['hits'][j]['_score'],
            'title': res[i]['hits']['hits'][j]['_source']['title'],
            'content': res[i]['hits']['hits'][j]['_source']['content']
        }
        passage = res[i]['hits']['hits'][j]['_source']['content']
        contents.append(content)
        passages.append(passage)
    test_datas.append({
        'id': test_dataset[i]['id'],
        'question': test_dataset[i]['question'],
        'contents': contents,
        'context': " ".join([p for p in passages])
    })

# a = pd.DataFrame(test_datas)
# a
Dataset.from_pandas(pd.DataFrame(test_datas)).save_to_disk(f"test_es_top_{k}.json")

,id,question,contents,context
0,mrc-1-000653,유령'은 어느 행성에서 지구로 왔는가?,"[{'docs_id': '43280', 'score': 20.487246, 'tit...",목성의 대기에서 보이는 줄무늬는 적도와 평행하면서 행성을 둘러싸는 대(zone)와 ...
1,mrc-1-001113,용병회사의 경기가 좋아진 것은 무엇이 끝난 이후부터인가?,"[{'docs_id': '20081', 'score': 23.784315, 'tit...",SK에서 방출된 이후 그는 일본을 여행하며 잠시 신변을 정리하고 있었다. 그 때 좌...
2,mrc-0-002191,돌푸스에게 불특정 기간동안 하원이 잠시 쉬는 것을 건의 받았던 인물은?,"[{'docs_id': '20554', 'score': 46.843483, 'tit...","1933년 3월, 투표 과정의 위법성에 대한 문제제기가 불거졌다. 당시 오스트리아 ..."
3,mrc-0-003951,"마오리언어와 영어, 뉴질랜드 수화를 공식 언어로 사용하는 나라는?","[{'docs_id': '25557', 'score': 61.7281, 'title...","유럽인들의 아메리카와 오세아니아 식민지화로 인해 아메리카와 오세아니아의 문화적, 민..."
4,mrc-1-001272,디엔비엔푸 전투에서 보응우옌잡이 상대한 국가는?,"[{'docs_id': '18335', 'score': 39.044395, 'tit...",1926년 학생 시절 베트남청년혁명당에 가입했고 1930년에 학생 파업을 지지했다는...
...,...,...,...,...
595,mrc-0-002989,타입 2 가이아 메모리을 만든 집단은?,"[{'docs_id': '26084', 'score': 35.11652, 'titl...","재단 X가 슈라우드(=소노자키 후미네)가 개발한 가이아 메모리의 테크놀로지를 분석,..."
596,mrc-0-001804,장면이 정치보복에 반대하는 입장에서 처벌을 원치 않은 대상은?,"[{'docs_id': '6969', 'score': 45.62529, 'title...",과거의 죄악에 대한 소급입법 적용을 반대하였다. 4.19 혁명 당시 사망한 사망자의...
597,mrc-0-003411,"콜드게임 중 어떠한 계기로 인해 잠시 중단된 뒤, 익일에 게임이 진행되는 것은?","[{'docs_id': '25696', 'score': 40.586067, 'tit...",콜드게임(called game)이란 야구나 소프트볼 등의 운동경기에서 심판에 의하여...
598,mrc-0-003436,제2캐나다기갑여단이 상륙한 곳은?,"[{'docs_id': '43341', 'score': 44.655956, 'tit...",주노 해변 주노 해변은 쿠르쇨르메르의 양쪽으로 뻗은 5마일 정도의 해변이었다. 제3...


In [98]:
Dataset.from_pandas(a).save_to_disk(f"test_es_top_{k}")

In [100]:
b = load_from_disk('test_es_top_20')
b

Dataset({
    features: ['id', 'question', 'contents', 'context'],
    num_rows: 600
})

In [105]:
b.column_names

['id', 'question', 'contents', 'context']

In [112]:
a.to_csv('h.csv')

In [109]:
from datasets import (load_from_disk, load_dataset, Features, 
                        Value, Sequence, DatasetDict, Dataset)
from tqdm import tqdm

test_dataset = load_from_disk('data/test_dataset/validation')

def make_test_dataset(res, err, k):
    if len(err) > 0:
        return

    test_datas = []
    for i in range(len(res)):
        contents = []
        passages = []
        for j in range(k):
            content = {
                'docs_id': res[i]['hits']['hits'][j]['_id'],
                'score': res[i]['hits']['hits'][j]['_score'],
                'title': res[i]['hits']['hits'][j]['_source']['title'],
                'content': res[i]['hits']['hits'][j]['_source']['content']
            }
            passage = res[i]['hits']['hits'][j]['_source']['content']
            contents.append(content)
            passages.append(passage)
        test_datas.append({
            'id': test_dataset[i]['id'],
            'question': test_dataset[i]['question'],
            'contents': contents,
            'context': " ".join([p for p in passages])
        })

    tmp_pd =pd.DataFrame(test_datas)
    Dataset.from_pandas(tmp_pd).save_to_disk(f"tt/test_es_top_{k}")
    print(f"top {k} dataset is made.")

def search_query(questions, size):
    error_queries = []
    results = []
    for question in questions:
        try:
            res = es.search(index=INDEX_NAME, q=question, size=size)
            results.append(res)
        except:
            error_queries.append(question)
    print(f"top {k} elastic search is finished")
    return results, error_queries


for k in tqdm([5, 7, 10, 11, 12, 13, 15, 17, 18]):
    res, err = search_query(test_query, k)
    make_test_dataset(res, err, k)


  0%|          | 0/9 [00:00<?, ?it/s]<ipython-input-109-152f200595e1>:41: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  res = es.search(index=INDEX_NAME, q=question, size=size)
 11%|█         | 1/9 [00:13<01:46, 13.32s/it]

top 5 elastic search is finished
top 5 dataset is made.
top 7 elastic search is finished


 22%|██▏       | 2/9 [00:27<01:35, 13.61s/it]

top 7 dataset is made.
top 10 elastic search is finished


 33%|███▎      | 3/9 [00:41<01:23, 13.95s/it]

...

top 15 dataset is made.
top 17 elastic search is finished


 89%|████████▉ | 8/9 [01:57<00:15, 15.26s/it]

top 17 dataset is made.
top 18 elastic search is finished


100%|██████████| 9/9 [02:13<00:00, 14.84s/it]

top 18 dataset is made.


In [110]:
c = load_from_disk('tt/test_es_top_20')
c

Dataset({
    features: ['id', 'question', 'contents', 'context'],
    num_rows: 600
})

In [184]:
train_dataset = load_from_disk('data/new_train_dataset/train')
train_query = train_dataset['question']
print(train_query[0], len(train_dataset))
train_dataset[0]

대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은? 3952


{'id': 'mrc-1-000067',
 'context': '미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다. 미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다. 미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05',
 'question': '대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?',
 'answers': {'text': ['하원'], 'answer_start': [229]}}

In [174]:
valid_dataset = load_from_disk('data/new_train_dataset/validation')
valid_query = valid_dataset['question']
print(valid_query[0], len(valid_dataset))
valid_dataset[0]

처음으로 부실 경영인에 대한 보상 선고를 받은 회사는? 240


{'id': 'mrc-0-003264',
 'context': '순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법시험에 합격하여 판사로 임용되었고 대법원 재판연구관, 수원지법 부장판사, 사법연수원 교수, 특허법원 부장판사 등을 거쳐 능력을 인정받았다. 2003년 최종영 대법원장의 지명으로 헌법재판소 재판관을 역임하였다. 경제민주화위원회(위원장 장하성이 소액주주들을 대표해 한보철강 부실대출에 책임이 있는 이철수 전 제일은행장 등 임원 4명을 상대로 제기한 손해배상청구소송에서 서울지방법원 민사합의17부는 1998년 7월 24일에 "한보철강에 부실 대출하여 은행에 막대한 손해를 끼친 점이 인정된다"며 "원고가 배상을 청구한 400억원 전액을 은행에 배상하라"고 하면서 부실 경영인에 대한 최초의 배상 판결을 했다. 2004년 10월 신행정수도의건설을위한특별조치법 위헌 확인 소송에서 9인의 재판관 중 유일하게 각하 견해를 내었다. 소수의견에서 전효숙 재판관은 다수견해의 문제점을 지적하면서 관습헌법 법리를 부정하였다. 전효숙 재판관은 서울대학교 근대법학교육 백주년 기념관에서 열린 강연에서, 국회가 고도의 정치적인 사안을 정치로 풀기보다는 헌법재판소에 무조건 맡겨서 해결하려는 자세는 헌법재판소에게 부담스럽다며 소회를 밝힌 바 있다.',
 'question': '처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?',
 'answers': {'text': ['한보철강'], 'answer_start': [281]}}

In [321]:
origin_train_dataset = load_from_disk('data/train_dataset')['train']
origin_train_dataset[0]

{'question': '대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?',
 'answers': {'answer_start': [235], 'text': ['하원']},
 'document_id': 18293,
 'id': 'mrc-1-000067',
 'title': '미국 상원',
 '__index_level_0__': 42,
 'context': '미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다.\\n\\n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.\\n\\n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안

In [356]:
origin_valid_dataset = load_from_disk('data/train_dataset')['validation']
origin_valid_dataset[0]

{'question': '처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?',
 'answers': {'answer_start': [284], 'text': ['한보철강']},
 'document_id': 9027,
 'id': 'mrc-0-003264',
 'title': '전효숙',
 '__index_level_0__': 2146,
 'context': '순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법시험에 합격하여 판사로 임용되었고 대법원 재판연구관, 수원지법 부장판사, 사법연수원 교수, 특허법원 부장판사 등을 거쳐 능력을 인정받았다. 2003년 최종영 대법원장의 지명으로 헌법재판소 재판관을 역임하였다.\\n\\n경제민주화위원회(위원장 장하성이 소액주주들을 대표해 한보철강 부실대출에 책임이 있는 이철수 전 제일은행장 등 임원 4명을 상대로 제기한 손해배상청구소송에서 서울지방법원 민사합의17부는 1998년 7월 24일에 "한보철강에 부실 대출하여 은행에 막대한 손해를 끼친 점이 인정된다"며 "원고가 배상을 청구한 400억원 전액을 은행에 배상하라"고 하면서 부실 경영인에 대한 최초의 배상 판결을 했다. \\n\\n2004년 10월 신행정수도의건설을위한특별조치법 위헌 확인 소송에서 9인의 재판관 중 유일하게 각하 견해를 내었다. 소수의견에서 전효숙 재판관은 다수견해의 문제점을 지적하면서 관습헌법 법리를 부정하였다. 전효숙 재판관은 서울대학교 근대법학교육 백주년 기념관에서 열린 강연에서, 국회가 고도의 정치적인 사안을 정치로 풀기보다는 헌법재판소에 무조건 맡겨서 해결하려는 자세는 헌법재판소에게 부담스럽다며 소회를 밝힌 바 있다.'}

In [360]:
def make_train_dataset(res, err, k):
    train_datas = []
    for i in range(len(res)):
        passages = []
        count = 0

        # contexts = train_dataset[i]['context']
        contexts = origin_train_dataset[i]['context']
        if len(res[i]['hits']['hits']) > 1:
            for j in range(k):
                if count == 1:
                    break
                if j == k - 1 and count < 1:
                    print('k늘려라.', count, i)
                    break

                passage = res[i]['hits']['hits'][j]['_source']['content']
                if passage != train_dataset[i]['context'] and train_dataset[i]['answers']['text'][0] not in passage:
                    contexts += ' ' + passage
                    count += 1

        train_datas.append({
            'id': train_dataset[i]['id'],
            'context': contexts,
            'question': train_dataset[i]['question'],
            # 'answers': train_dataset[i]['answers']
            'answers': origin_train_dataset[i]['answers']
        })

    tmp_pd =pd.DataFrame(train_datas)
    Dataset.from_pandas(tmp_pd).save_to_disk(f"not_include_answer_passage_train_es_top_1")
    print(f"top {k} dataset is made.")
    return tmp_pd

# def make_train_dataset(res, err, k):
#     train_datas = []
#     for i in range(len(res)):
#         passages = []

#         contexts = train_dataset[i]['context']
#         if len(res[i]['hits']['hits']) > 1:
#             for j in range(k - 1):
#                 passage = res[i]['hits']['hits'][j]['_source']['content']
#                 if passage != train_dataset[i]['context']:
#                     contexts += ' ' + passage

#         train_datas.append({
#             'id': train_dataset[i]['id'],
#             'context': contexts,
#             'question': train_dataset[i]['question'],
#             'answers': train_dataset[i]['answers']
#         })

#     tmp_pd =pd.DataFrame(train_datas)
#     Dataset.from_pandas(tmp_pd).save_to_disk(f"train_es_top_{k}")
#     print(f"top {k} dataset is made.")
#     return tmp_pd

# def make_valid_dataset(res, err, k):
#     valid_datas = []
#     for i in range(len(res)):
#         passages = []

#         contexts = valid_dataset[i]['context']
#         if len(res[i]['hits']['hits']) > 1:
#             for j in range(k - 1):
#                 passage = res[i]['hits']['hits'][j]['_source']['content']
#                 if passage != valid_dataset[i]['context']:
#                     contexts += ' ' + passage

#         valid_datas.append({
#             'id': valid_dataset[i]['id'],
#             'context': contexts,
#             'question': valid_dataset[i]['question'],
#             'answers': valid_dataset[i]['answers']
#         })

#     tmp_pd =pd.DataFrame(valid_datas)
#     Dataset.from_pandas(tmp_pd).save_to_disk(f"valid_es_top_{k}")
#     print(f"top {k} dataset is made.")
#     return tmp_pd

def make_valid_dataset(res, err, k):
    valid_datas = []
    for i in range(len(res)):
        passages = []
        count = 0

        contexts = origin_valid_dataset[i]['context']
        if len(res[i]['hits']['hits']) > 1:
            for j in range(k):
                if count == 19:
                    break
                if j == k - 1 and count < 19:
                    print('k늘려라.', count, i)
                    break

                passage = res[i]['hits']['hits'][j]['_source']['content']
                if passage != valid_dataset[i]['context'] and valid_dataset[i]['answers']['text'][0] not in passage:
                    contexts += ' ' + passage
                    count += 1

        valid_datas.append({
            'id': valid_dataset[i]['id'],
            'context': contexts,
            'question': valid_dataset[i]['question'],
            'answers': origin_valid_dataset[i]['answers']
        })

    tmp_pd =pd.DataFrame(valid_datas)
    Dataset.from_pandas(tmp_pd).save_to_disk(f"not_include_answer_passage_valid_es_top_19")
    print(f"top {k} dataset is made.")
    return tmp_pd


def search_query(questions, size):
    error_queries = []
    results = []
    for question in questions:
        if question == "11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?":
            question = "11\~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?"
        if question == "S/W 취약점 찾기 대회에서 수상한 해커그룹들이 찾은 보안 취약점의 특징은?":
            question = "S\/W 취약점 찾기 대회에서 수상한 해커그룹들이 찾은 보안 취약점의 특징은?"
        if question == "병에 걸려 죽을 확률이 약 25~50%에 달하는 유형의 질병은?":
            question = "병에 걸려 죽을 확률이 약 25\~50\%에 달하는 유형의 질병은?'"
        try:
            res = es.search(index=INDEX_NAME, q=question, size=size)
            results.append(res)
        except:
            error_queries.append(question)
    print(f"top {k} elastic search is finished")
    return results, error_queries


for k in tqdm([30]):
# for k in tqdm([5, 10, 15, 20]):
    res, err = search_query(train_query, k)
    print(f"len_train: {len(res)} , len_error: {len(err)}")
    f = make_train_dataset(res, err, k)
    
    # res, err = search_query(valid_query, k)
    # print(f"len_valid: {len(res)} , len_error: {len(err)}")
    # f = make_valid_dataset(res, err, k)

f

  0%|          | 0/1 [00:00<?, ?it/s]

len_valid: 240 , len_error: 0


100%|██████████| 1/1 [00:01<00:00,  1.23s/it]

top 100 dataset is made.


,id,context,question,answers
0,mrc-0-003264,"순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법...",처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?,"{'answer_start': [284], 'text': ['한보철강']}"
1,mrc-0-004762,요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & ...,스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?,"{'answer_start': [146], 'text': ['1871년']}"
2,mrc-1-001810,"촐라 정부\n 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 ...",촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?,"{'answer_start': [517], 'text': ['나뭇잎']}"
3,mrc-1-000219,프랑스의 십자군 무훈시는 1099년 예루살렘 왕국의 통치자가 된 고드프루아 드 부용...,로타이르가 백조를 구하기 위해 사용한 것은?,"{'answer_start': [1109], 'text': ['금대야']}"
4,mrc-1-000285,탈관료제화는 현대사회에서 관료제 성격이 약화되는 현상이다. 현대사회에서 관료제는 약...,의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?,"{'answer_start': [386], 'text': ['수평적 관계']}"
...,...,...,...,...
235,mrc-0-000484,"연나라 군대의 사령관이 악의에서 기겁으로 교체되자, 전단은 스스로 신령의 계시를 받...",전단이 연나라와의 전쟁에서 승리했을 당시 제나라의 왕은 누구인가?,"{'answer_start': [1084], 'text': ['제 양왕']}"
236,mrc-0-002095,현재 우리가 볼 수 있는 티칼의 모습은 펜실베이니아 대학교와 과테말라 정부의 협조 ...,공놀이 경기장 중 일부는 어디에 위치하고 있나?,"{'answer_start': [343], 'text': [''일곱 개의 신전 광장..."
237,mrc-0-003083,1940년 5월 1일 오전 창씨개명에 비협조적이라는 이유로 조선총독부 경무국에서 소...,창씨개명령의 시행일을 미루는 것을 수락한 인물은?,"{'answer_start': [247], 'text': ['미나미 지로']}"
238,mrc-0-002978,빌카밤바 지역은 파차쿠티 황제 때 부터 잉카 제국에 속해있던 지역이었다. 스페인 군...,망코 잉카가 쿠스코를 되찾기 위해 마련한 군사는 총 몇 명인가?,"{'answer_start': [563], 'text': ['200,000명']}"


In [355]:
len(res)

3952

In [274]:
print(len(f['context'][0]), len(valid_dataset['context'][0]))

18391 618


In [259]:
print(len(f['context'][2212]), len(train_dataset['context'][2212]))

16970 705


In [125]:
Dataset.from_pandas(f).save_to_disk(f"train_es_top_5")

In [196]:
!tar -cvf test_es_top_20.tar 'tt/test_es_top_20'

tt/test_es_top_20/
tt/test_es_top_20/cache-04b809be74276a1f.arrow
tt/test_es_top_20/cache-9486682dcd40adb0.arrow
tt/test_es_top_20/state.json
tt/test_es_top_20/cache-1a991d41b3904672.arrow
tt/test_es_top_20/dataset_info.json
tt/test_es_top_20/cache-6b0bc50ab3aa12cd.arrow
tt/test_es_top_20/dataset.arrow


In [182]:
!cp 'data/new_train_dataset/dataset_dict.json' 'concat_top_5'

In [187]:
!mkdir concat_top_20

In [189]:
!cp 'data/new_train_dataset/dataset_dict.json' 'concat_top_20'
!cp -rf 'concat_top_5/validation' 'concat_top_20'

In [192]:
!tar -cvf concat_top_20.tar concat_top_20

concat_top_20/
concat_top_20/dataset_dict.json
concat_top_20/validation/
concat_top_20/validation/state.json
concat_top_20/validation/dataset_info.json
concat_top_20/validation/dataset.arrow
concat_top_20/.ipynb_checkpoints/
concat_top_20/train/
concat_top_20/train/state.json
concat_top_20/train/dataset_info.json
concat_top_20/train/dataset.arrow


In [275]:
!tar -cvf valid_es_top_20.tar valid_es_top_20

valid_es_top_20/
valid_es_top_20/state.json
valid_es_top_20/dataset_info.json
valid_es_top_20/dataset.arrow


In [262]:
!tar -cvf test_es_top_5.tar 'tt/test_es_top_5'

tt/test_es_top_5/
tt/test_es_top_5/state.json
tt/test_es_top_5/dataset_info.json
tt/test_es_top_5/cache-771a3b4aabd2c649.arrow
tt/test_es_top_5/dataset.arrow


In [334]:
!mkdir train_with_origin_gt_add_top_k_passage

In [335]:
!cp -rf not_* train_with_origin_gt_add_top_k_passage

In [336]:
!tar -cvf train_with_origin_gt_add_top_k_passage.tar train_with_origin_gt_add_top_k_passage

train_with_origin_gt_add_top_k_passage/
train_with_origin_gt_add_top_k_passage/not_include_answer_passage_train_es_top_19/
train_with_origin_gt_add_top_k_passage/not_include_answer_passage_train_es_top_19/state.json
train_with_origin_gt_add_top_k_passage/not_include_answer_passage_train_es_top_19/dataset_info.json
train_with_origin_gt_add_top_k_passage/not_include_answer_passage_train_es_top_19/dataset.arrow
train_with_origin_gt_add_top_k_passage/not_include_answer_passage_train_es_top_4/
train_with_origin_gt_add_top_k_passage/not_include_answer_passage_train_es_top_4/state.json
train_with_origin_gt_add_top_k_passage/not_include_answer_passage_train_es_top_4/dataset_info.json
train_with_origin_gt_add_top_k_passage/not_include_answer_passage_train_es_top_4/dataset.arrow
train_with_origin_gt_add_top_k_passage/not_include_answer_passage_train_es_top_9/
train_with_origin_gt_add_top_k_passage/not_include_answer_passage_train_es_top_9/state.json
train_with_origin_gt_add_top_k_passage/not_inc

In [354]:
!tar -xvf test_es_top_20.tar

tt/test_es_top_20/
tt/test_es_top_20/cache-04b809be74276a1f.arrow
tt/test_es_top_20/cache-9486682dcd40adb0.arrow
tt/test_es_top_20/state.json
tt/test_es_top_20/cache-1a991d41b3904672.arrow
tt/test_es_top_20/dataset_info.json
tt/test_es_top_20/cache-6b0bc50ab3aa12cd.arrow
tt/test_es_top_20/dataset.arrow


In [361]:
b1 = load_from_disk('train_with_origin_gt_add_top_k_passage/not_include_answer_passage_train_es_top_4')
b2 = load_from_disk('valid_with_origin_gt_add_top_k_passage/not_include_answer_passage_valid_es_top_4')

In [362]:
b1

Dataset({
    features: ['id', 'context', 'question', 'answers'],
    num_rows: 3952
})

In [363]:
b2

Dataset({
    features: ['id', 'context', 'question', 'answers'],
    num_rows: 240
})

In [366]:
b3 = datasets.concatenate_datasets([b1, b2])
b3

Dataset({
    features: ['id', 'context', 'question', 'answers'],
    num_rows: 4192
})

In [386]:
test_dataset[0]

{'id': 'mrc-1-000653', 'question': "유령'은 어느 행성에서 지구로 왔는가?"}

In [369]:
import pickle

with open('hybrid_retrieval_test.bin','rb') as f:
    tett = pickle.load(f)

In [381]:
t_q = list(tett.keys())
t_p = list(tett.values())

In [391]:
len(t_p)

600

In [393]:
hb_t = []

for i in range(len(t_q)):
    hb_t.append({
        'id': test_dataset['id'][i],
        'question': t_q[i],
        'context': " ".join([p for p in t_p[i][:20]])
    })

600

In [395]:
hb =pd.DataFrame(hb_t)
Dataset.from_pandas(hb).save_to_disk(f"hb_k_20")

In [396]:
nn = load_from_disk('hb_k_20')
nn

Dataset({
    features: ['id', 'question', 'context'],
    num_rows: 600
})

In [400]:
!tar -cvf hb_k_20.tar hb_k_20

hb_k_20/
hb_k_20/state.json
hb_k_20/dataset_info.json
hb_k_20/cache-4be93ff1a11523b7.arrow
hb_k_20/dataset.arrow


In [16]:
import pickle

with open('hybrid_train_retrieval.bin','rb') as f:
    tett = pickle.load(f)
    
t_q = list(tett.keys())
t_p = list(tett.values())

In [21]:
len(t_q)

3952

In [22]:
len(t_p[0])

100

In [23]:
train_dataset[0]

{'answers': {'text': ['하원'], 'answer_start': [229]},
 'id': 'mrc-1-000067',
 'question': '대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?',
 'context': '미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다. 미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다. 미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05'}

In [24]:
origin_train_dataset = load_from_disk('data/train_dataset')['train']

In [25]:
train_datas = []
for i in range(len(t_q)):
    passages = []
    count = 0

    # contexts = train_dataset[i]['context']
    contexts = origin_train_dataset[i]['context']
    for j in range(100):
        if count == 4:
            break
        if j == 99 and count < 4:
            print('k늘려라.', count, i)
            break

        passage = t_p[i][j]
        if passage != train_dataset[i]['context'] and train_dataset[i]['answers']['text'][0] not in passage:
            contexts += ' ' + passage
            count += 1

    train_datas.append({
        'id': train_dataset[i]['id'],
        'context': contexts,
        'question': train_dataset[i]['question'],
        # 'answers': train_dataset[i]['answers']
        'answers': origin_train_dataset[i]['answers']
    })

tmp_pd =pd.DataFrame(train_datas)
Dataset.from_pandas(tmp_pd).save_to_disk(f"hb_train_k_5")

In [26]:
nn = load_from_disk('hb_train_k_5')
nn

Dataset({
    features: ['id', 'context', 'question', 'answers'],
    num_rows: 3952
})

In [28]:
!tar -cvf hb_train_k_5.tar hb_train_k_5

hb_train_k_5/
hb_train_k_5/state.json
hb_train_k_5/dataset_info.json
hb_train_k_5/dataset.arrow


In [29]:
len(nn[0]['context'])

3433

In [414]:
qg = pd.read_csv('preprob_qg.csv')
qg.head(1)

,Unnamed: 0,index,Unnamed: 0.1,context,question,answer,sts_score,mrc_answers,context_mapping_qg_to_origin
0,0,0,1,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,계사명 사리구는 경상남도 울주군 어느면에서 출토되었다고 전해지는가?,웅촌면,0.418,"('웅촌면', (45, 48))",3237


In [44]:
qg=pd.read_csv('qg_filter.csv')
qg.head(1)

,Unnamed: 0,level_0,Unnamed: 0.1,index,Unnamed: 0.1.1,context,question,answer,sts_score,mrc_answers,context_mapping_qg_to_origin
0,0,2,2,2,5,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,계사명 사리구는 세로로 몇 줄 쓰여있어?,세 줄,0.618,"('세 줄', (132, 134))",3237


In [45]:
qg_query = list(qg['question'])

In [46]:
origin_train_dataset[int(qg['context_mapping_qg_to_origin'][0])]

{'__index_level_0__': 361,
 'document_id': 47334,
 'title': '계사명 사리구',
 'answers': {'answer_start': [187], 'text': ['청화안료']},
 'question': '승표 부부에 대해 적혀있는 명문은 무엇으로 쓰여진 것인가?',
 'id': 'mrc-0-000533',
 'context': '동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기의 유물로, 경상남도 울주군 웅촌면 대복리에서 출토되었다고 전한다. 외함(外函)은 청화명문이 있는 백자이며, 그 안쪽에 납작한 금속제 원형 합 2점과 금속제 원통형 합 등 3점의 그릇이 봉안되어 있다.\\n\\n바깥쪽의 외함인 백자 합 동체 중앙부 표면에 청화안료로 쓴 “癸巳二月日 施主承表 兩主”라는 명문이 세로로 세 줄에 걸쳐서 쓰여 있어 조선 후기인 계사년에 시주자인 승표 부부가 발원하여 만든 것임을 알 수 있다.\\n\\n동아대학교박물관의 계사명 사리구는 정확한 제작연대는 알 수 없지만 명문 등을 통해 적어도 17세기 이후에 제작된 것으로 추정되는 작품으로, 명문이 있는 조선 후기 경상도 지역 출토 사리장엄구라는 점에서 중요한 가치를 지닌 작품으로 판단된다.\\n\\n조선 후기 사리장엄구는 아직까지 조사와 연구가 거의 이루어지지 않았으나, 이처럼 세트를 갖추어 출토된 유물은 비교적 드문 편임을 고려할 때, 이 계사명 사리장엄구는 제작연대와 발원자의 이름이 밝혀져 있으며, 지금까지 출토된 예가 드문 비교적 완전한 세트를 가진 유물이라는 점에서 조선 후기 사리장엄구 연구에 자료적 가치를 지닌 유물이다.'}

In [47]:
def make_qg_dataset(res, err, k):
    qg_datas = []
    idx = []
    for i in range(len(res)):
        passages = []
        count = 0

        contexts = origin_train_dataset[int(qg['context_mapping_qg_to_origin'][i])]['context']
        if len(res[i]['hits']['hits']) > 1:
            for j in range(k):
                if count == 4:
                    break
                if j == k - 1 and count < 4:
                    print('k늘려라.', count, i)
                    idx.append(i)
                    break

                passage = res[i]['hits']['hits'][j]['_source']['content']
                if passage != qg['context'][i] and qg['answer'][i] not in passage:
                    contexts += ' ' + passage
                    count += 1

        qg_datas.append({
            'id': 'mrc-qg-' + str(qg['index'][i]),
            'context': contexts,
            'question': qg['question'][i],
            'answers': {
                'text': [qg['answer'][i]],
                'answer_start': [int(eval(qg['mrc_answers'][i])[1][0])]
            }
        })

    tmp_pd =pd.DataFrame(qg_datas)
    # Dataset.from_pandas(tmp_pd).save_to_disk(f"qg_dataset_top_4")
    print(f"top 5 concat qg dataset is made.")
    return tmp_pd, idx


def search_query(questions, size):
    error_queries = []
    results = []
    for question in questions:
        if question == '궁전 중정이 있는 자리에 25명~50명 규모의 마을이 있던 시기는?':
            question = '궁전 중정이 있는 자리에 25명\~50명 규모의 마을이 있던 시기는?'
        if question =='오늘날에도 볼 수 있듯 한 마을은 여러 이것으로 이뤄졌으며 서로 관계를 맺었고 일종의 족외혼 관습을 가졌다. 이들은 폐쇄된 공간에서 살았는데 그 안에서 사생활은 거의 없으며 매우 가깝게 생활했다. 선토기 신석기 시대인 기원전 7000년경에 궁전 중정이 있는 자리에는 25명~50명 규모의 마을이 있었는데 이들은 초가집을 짓고 가축을 키우고 작물을 재배했으며 불상사가 닥치면 유아를 바닥에 묻은것을 무엇이라고 하는가?':
            question = '오늘날에도 볼 수 있듯 한 마을은 여러 이것으로 이뤄졌으며 서로 관계를 맺었고 일종의 족외혼 관습을 가졌다. 이들은 폐쇄된 공간에서 살았는데 그 안에서 사생활은 거의 없으며 매우 가깝게 생활했다. 선토기 신석기 시대인 기원전 7000년경에 궁전 중정이 있는 자리에는 25명\~50명 규모의 마을이 있었는데 이들은 초가집을 짓고 가축을 키우고 작물을 재배했으며 불상사가 닥치면 유아를 바닥에 묻은것을 무엇이라고 하는가?'
        if question == '200명~600명 규모의 마을이 크노소스 궁전 터와 북쪽 및 서쪽 언덕 비탈 대부분을 점유한 시기는?':
            question = '200명\~600명 규모의 마을이 크노소스 궁전 터와 북쪽 및 서쪽 언덕 비탈 대부분을 점유한 시기는?'
        if question == '신석기 초기(기원전 6000년~5000년) 크노소스 궁전 터와 북쪽 및 서쪽 언덕 비탈 대부분을 점유하던 집단의 규모는?':
            question = '신석기 초기(기원전 6000년\~5000년) 크노소스 궁전 터와 북쪽 및 서쪽 언덕 비탈 대부분을 점유하던 집단의 규모는?'
        if question == '선토기 신석기 시대인 기원전 7000년~6000년경에 궁전 중정이 있는 자리에는 몇명의 마을이 있었는가?':
            question = '선토기 신석기 시대인 기원전 7000년\~6000년경에 궁전 중정이 있는 자리에는 몇명의 마을이 있었는가?'
        if question == '신석기 초기의 200명~600명 규모의 마을이 크노소스 궁전 터와 북쪽 그리고 어느쪽 언덕 비탈 대부분을 점유했나?':
            question = '신석기 초기의 200명\~600명 규모의 마을이 크노소스 궁전 터와 북쪽 그리고 어느쪽 언덕 비탈 대부분을 점유했나?'
        if question == '기원전 7000년~6500년경으로 거슬러 올라가는 것은 연대측정법에서 무엇을 측정하기 때문인가?':
            question = '기원전 7000년\~6500년경으로 거슬러 올라가는 것은 연대측정법에서 무엇을 측정하기 때문인가?'
        if question == '기원전 7000년~6000년경이 무슨 시대인가?':
            question ='기원전 7000년\~6000년경이 무슨 시대인가?'
        if question == '기원전 7000년~6500년경 크레타에서는 진흙이나 돌에 새겨진 실감개와 어떤 무늬가 많이 출토되어 옷을 지어 입었음을 알 수 있는가?':
            question = '기원전 7000년\~6500년경 크레타에서는 진흙이나 돌에 새겨진 실감개와 어떤 무늬가 많이 출토되어 옷을 지어 입었음을 알 수 있는가?'
        if question == '슬라브조어/공통슬라브어 시기는 크게 몇 단계로 구분할 수 있나?':
            question = '슬라브조어\/공통슬라브어 시기는 크게 몇 단계로 구분할 수 있나?'
        if question == '2년마다 어느 주에서 1/3씩 상원의원을 새로 선출하여 연방에 보내는가?':
            question = '2년마다 어느 주에서 1\/3씩 상원의원을 새로 선출하여 연방에 보내는가?'
        if question == '단추형의 3V 리튬 전지로서 20mm의 지름에 1.6~4mm의 두께를 가지는 것은?':
            question = '단추형의 3V 리튬 전지로서 20mm의 지름에 1.6\~4mm의 두께를 가지는 것은?'
        if question == '8~9세기에 잉글랜드어를 북게르만어군의 영향을 받게 한 침입자는?':
            question = '8\~9세기에 잉글랜드어를 북게르만어군의 영향을 받게 한 침입자는?'
        if question == '국부의 뿌리까지 모두 제거하는 수술을 시도한 환관지망생은 80~90%의 확률로 사망했는데 이는 어떤 수술 전의 체력이 보통사람에 비해 월등했다고 추측되는가?':
            question = '국부의 뿌리까지 모두 제거하는 수술을 시도한 환관지망생은 80\~90\%의 확률로 사망했는데 이는 어떤 수술 전의 체력이 보통사람에 비해 월등했다고 추측되는가?'
        if question == '바순에 중점을 둔 안단테의 3/4박자는?':
            question = '바순에 중점을 둔 안단테의 3\/4박자는?'
        if question == '안단테 가장조 3/4박자에서 클라리넷은 몇 분 음표로 펼침화음적 상하행을 반복하는가?':
            question = '안단테 가장조 3\/4박자에서 클라리넷은 몇 분 음표로 펼침화음적 상하행을 반복하는가?'
        if question == '안단테 가장조 3/4박자는 앞 악장과는 달리 무엇에 중점을 두고있는가?':
            question = '안단테 가장조 3\/4박자는 앞 악장과는 달리 무엇에 중점을 두고있는가?'
        if question == '이마나가가 상시 140km/h대를 측정한 게 몇 학년 때야?':
            question = '이마나가가 상시 140km\/h대를 측정한 게 몇 학년 때야?'
        if question == '2000년 김성근은 MBC/LG 선수 출신 최초로 LG트윈스에서 어떤 직책을 맡았는가?':
            question = '2000년 김성근은 MBC\/LG 선수 출신 최초로 LG트윈스에서 어떤 직책을 맡았는가?'
        if question == '뇌전증환자의 30~40%는 장기간 발작이 없는 경우 무엇을 중단해도 재발이 없는 완치 상태가 되는가?':
            question = '뇌전증환자의 30\~40\%는 장기간 발작이 없는 경우 무엇을 중단해도 재발이 없는 완치 상태가 되는가?'
        if question == '마우리츠는 9월 28~29일 어디로 진격해 소집군인들을 해산시켰는가?':
            question = '마우리츠는 9월 28\~29일 어디로 진격해 소집군인들을 해산시켰는가?'
        if question == '마리암 여왕이 1029/1930년에 방문한 곳은 어디인가?':
            question = '마리암 여왕이 1029\/1930년에 방문한 곳은 어디인가?'
        if question == '김동한의 사망 몇 주년을 기념한 동상/기념비 제막식이 열렸는가?':
            question = '김동한의 사망 몇 주년을 기념한 동상\/기념비 제막식이 열렸는가?'
        if question == '김동한의 사망 2주기를 기념한 동상/기념비 제막식에서 좌담회 사회를 맡은 사람은?':
            question = '김동한의 사망 2주기를 기념한 동상\/기념비 제막식에서 좌담회 사회를 맡은 사람은?'
        if question == '론도 알레그로 마 논 트로포 바장조 6/8박자에서 마지막에 유니즌과 함께 힘차게 모티브A를 반복하는 악기는?':
            question = '론도 알레그로 마 논 트로포 바장조 6\/8박자에서 마지막에 유니즌과 함께 힘차게 모티브A를 반복하는 악기는?'
        if question == '론도 알레그로 마 논 트로포 바장조 6/8박자는 서두 몇 마디에서 제시되는 소재가 악장을 완전히 지배하는가?':
            question = '론도 알레그로 마 논 트로포 바장조 6\/8박자는 서두 몇 마디에서 제시되는 소재가 악장을 완전히 지배하는가?'
        if question == '론도 알레그로 마 논 트로포 바장조 6/8박자에서 A의 반행형을 연주하는 악기는?':
            question = '론도 알레그로 마 논 트로포 바장조 6\/8박자에서 A의 반행형을 연주하는 악기는?'
        if question == '론도 알레그로 마 논 트로포 바장조 6/8박자에서 4마디에서 제시되는 소재가 악장을 완전히 지배하고 있는 부분은?':
            question = '론도 알레그로 마 논 트로포 바장조 6\/8박자에서 4마디에서 제시되는 소재가 악장을 완전히 지배하고 있는 부분은?'
        if question == '1940년대와 50년대 초에 들어서면서 베이/블루어가 어느 노선 못지 않게 주요 환승 지점으로 발돋움했는가?':
            question = '1940년대와 50년대 초에 들어서면서 베이\/블루어가 어느 노선 못지 않게 주요 환승 지점으로 발돋움했는가?'
        if question == '적외선 뱃머리 충격파가 100km/s의 온도에서 이동하는 가스의 속도는?':
            question = '적외선 뱃머리 충격파가 100km\/s의 온도에서 이동하는 가스의 속도는?'
        if question == 'Arch. Simancas, E° Flandes, 559. }}는 몇년도에 발표된 것인가?':
            question = 'Arch. Simancas, E\° Flandes, 559. \}\}는 몇년도에 발표된 것인가?'
        if question == '비타민 A는 몇년에 2~3번 주어야 하는가?':
            question = '비타민 A는 몇년에 2\~3번 주어야 하는가?'
        if question == '삼척기줄다리기는 차츰 규모가 커지면서 7~8일 경이면 무슨 줄다리기 운동으로 번지나?':
            question = '삼척기줄다리기는 차츰 규모가 커지면서 7\~8일 경이면 무슨 줄다리기 운동으로 번지나?'
        if question == '자신의 죽기전 병석에서 힐에게 [전투준비]를 요구한 사람은?':
            question = '자신의 죽기전 병석에서 힐에게 \[전투준비\]를 요구한 사람은?'
        if question == '2003년 운정1동~3동 지역을 지정하여 개발하기 시작한 곳은 어디인가?':
            question = '2003년 운정1동\~3동 지역을 지정하여 개발하기 시작한 곳은 어디인가?'
        if question == '정부가 2기 신도시 개발 지역으로 운정1동~3동 지역을 지정한 해는?':
            question = '정부가 2기 신도시 개발 지역으로 운정1동\~3동 지역을 지정한 해는?'
        if question == '대한민국의 은둔형 외톨이가 2~30만 명 존재한다는 보고서가 나온 해는?':
            question = '대한민국의 은둔형 외톨이가 2\~30만 명 존재한다는 보고서가 나온 해는?'
        if question == '2008년 보고서에 따르면 어느 나라에 은둔형 외톨이가 2~30만 명 존재한다고 나타났는가?':
            question = '2008년 보고서에 따르면 어느 나라에 은둔형 외톨이가 2\~30만 명 존재한다고 나타났는가?'
        if question == '이륙 활주시 추력 레버 TO/GA 스위치를 누른 사람은?':
            question = '이륙 활주시 추력 레버 TO\/GA 스위치를 누른 사람은?'
        if question == '1904년 2월 8일~9일 밤, 여순항에서 러시아 함대에 대한 기습 공격을 시작한 군대는?':
            question = '1904년 2월 8일\~9일 밤, 여순항에서 러시아 함대에 대한 기습 공격을 시작한 군대는?'
        try:
            res = es.search(index=INDEX_NAME, q=question, size=size)
            results.append(res)
        except:
            error_queries.append(question)
    print(f"top {k} elastic search is finished")
    return results, error_queries


for k in tqdm([100]):
    res, err = search_query(qg_query, k)
    print(f"len_qg: {len(res)} , len_error: {len(err)}")
    f, iiii2 = make_qg_dataset(res, err, k)

# print(len(res), len(err))
# f
print(iiii2)

  0%|          | 0/1 [00:00<?, ?it/s]<ipython-input-47-238a70ea448a>:124: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  res = es.search(index=INDEX_NAME, q=question, size=size)


top 100 elastic search is finished
len_qg: 1625 , len_error: 0
k늘려라. 2 208
k늘려라. 0 297
k늘려라. 0 727
k늘려라. 2 833
k늘려라. 3 855


100%|██████████| 1/1 [01:25<00:00, 85.96s/it]

top 5 concat qg dataset is made.
[208, 297, 727, 833, 855]


In [48]:
iiii3 = [i for i in range(len(f)) if i not in iiii2]
len(iiii3)

1620

In [49]:
len(iiii2)

5

In [50]:
qgqg = f.loc[iiii3]
# ii = [i for i in range(len(qgqg))]
ii = list(qgqg['id'])
cc = list(qgqg['context'])
qqu = list(qgqg['question'])
aan = list(qgqg['answers'])
qgqg2 = pd.DataFrame(data={'answers': aan, 'context': cc, 'id': ii, 'question': qqu})
qgqg2

,answers,context,id,question
0,"{'text': ['세 줄'], 'answer_start': [132]}",동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,mrc-qg-2,계사명 사리구는 세로로 몇 줄 쓰여있어?
1,"{'text': ['베토벤'], 'answer_start': [83]}","자신의 후견자 겸 제자인 루돌프 대공을 위해, 1809년에 프랑스 군대의 포격이 쏟...",mrc-qg-14,교향곡 다장조를 작곡한 사람이 누구야?
2,"{'text': ['삼국유사'], 'answer_start': [33]}",고조선의 도읍지는 여러 차례 이동한 것으로 기록되어 있다. 《삼국유사》는 《고기》를...,mrc-qg-34,단군왕검이 1천 5백년간 나라를 다스렸다고 기록된 책은?
3,"{'text': ['5 mm'], 'answer_start': [200]}",너비아니(nubianie)는 궁중식 불고기로 알려진 한국 요리이다. 쇠고기를 칼로 ...,mrc-qg-53,쇠고기의 두께를 알려줘?
4,"{'text': ['삼손'], 'answer_start': [13]}",삼손은 단 지파의 히브리인으로 그의 어머니 하술렐보니(Hazelelponit)에 의...,mrc-qg-67,나실인이 되기로 서약된 사람은?
...,...,...,...,...
1615,"{'text': ['국민'], 'answer_start': [162]}",1947년 6월 미군정 사령관 존 하지는 이승만에게 이승만과 김구가 계획 중이라는 ...,mrc-qg-23783,여운형이 최우선으로 생각했던 것은?
1616,"{'text': ['남쪽'], 'answer_start': [83]}",전날 밤부터 난지도쪽으로 한강을 도하한 1개 연대 규모의 북한군이 오류동 정면으로 ...,mrc-qg-23784,북한군이 공격에 앞서 비행장쪽에서 그들의 보병에 화력을 지원하던 전차 2대를 야음을...
1617,"{'text': ['소련'], 'answer_start': [64]}",1969년에 연정에 합의한 사민당과 자유민주당의 사회자유 연정은 기존 외교 정책에서...,mrc-qg-23785,1969년 사민당과 자유민주당의 사회자유 연정이 우호적 입장을 취한 나라는 동독과 ...
1618,"{'text': ['미국'], 'answer_start': [37]}",오이 샌드위치는 영국이 기원인 샌드위치의 하나이다. 현대의 다른 종류의 오이샌드위치...,mrc-qg-23786,현대의 오이샌드위치는 대부분 어느 나라에서 만들어지나요?


In [53]:
from datasets import (load_from_disk, load_dataset, Features, 
                        Value, Sequence, DatasetDict, Dataset)
from tqdm import tqdm

fs = Features({'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int64', id=None)}, length=-1, id=None),
                'context': Value(dtype='string', id=None),
                'id': Value(dtype='string', id=None),
                'question': Value(dtype='string', id=None)})
Dataset.from_pandas(qgqg2, features=fs).save_to_disk(f"qg_filters")

In [483]:
!tar -cvf qg_dataset_top_4.tar qg_dataset_top_4

qg_dataset_top_4/
qg_dataset_top_4/state.json
qg_dataset_top_4/dataset_info.json
qg_dataset_top_4/dataset.arrow


In [54]:
bba = load_from_disk('qg_filters')
bba

Dataset({
    features: ['answers', 'context', 'id', 'question'],
    num_rows: 1620
})

In [55]:
train_dataset

Dataset({
    features: ['answers', 'context', 'id', 'question'],
    num_rows: 3952
})

In [56]:
print(bba[0]['id'], train_dataset[0]['id'])

mrc-qg-2 mrc-1-000067


In [57]:
h1h = load_from_disk('train_with_origin_gt_add_top_k_passage/not_include_answer_passage_train_es_top_4')

In [58]:
h1h

Dataset({
    features: ['id', 'context', 'question', 'answers'],
    num_rows: 3952
})

In [59]:
iis = []
ccs = []
qqs = []
aas = []

iis = h1h['id']
ccs = h1h['context']
qqs = h1h['question']
aas = h1h['answers']
qgqg3 = pd.DataFrame(data={'answers': aas, 'context': ccs, 'id': iis, 'question': qqs})
qgqg3

,answers,context,id,question
0,"{'answer_start': [235], 'text': ['하원']}",미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,mrc-1-000067,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?
1,"{'answer_start': [212], 'text': ['《경영의 실제》']}",'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,mrc-0-004397,현대적 인사조직관리의 시발점이 된 책은?
2,"{'answer_start': [510], 'text': ['백성']}",강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,mrc-1-000362,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?
3,"{'answer_start': [625], 'text': ['중국']}","불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",mrc-0-001510,11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?
4,"{'answer_start': [30], 'text': ['4개']}",동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,mrc-0-000823,명문이 적힌 유물을 구성하는 그릇의 총 개수는?
...,...,...,...,...
3947,"{'answer_start': [317], 'text': ['지질 구조']}",이오의 산\n이오의 산 목록\n 이오에는 100~150개의 산이 있다. 이들 산의 ...,mrc-0-005285,대부분의 이오의 산이 형성되는데 영향을 끼친 것은?
3948,"{'answer_start': [616], 'text': ['뉴저지']}",애니의 고군분투 뉴욕 입성기!!\n\n인류학자가 꿈인 21살 소녀 '애니(스칼렛 요...,mrc-1-000699,애니는 어디서 태어났는가?
3949,"{'answer_start': [255], 'text': ['칼 에케르트']}",1842년에 작곡가이자 지휘자인 오토 니콜라이가 빈 궁정 오페라극장 소속 관현악단을...,mrc-0-003429,1854년에 니콜라이의 뒤를 이어 상임 지휘자로서 활동한 인물은?
3950,"{'answer_start': [476], 'text': ['슬라브 신화']}",원어는 고대 그리스어까지 거슬러 올라간다. 영어 문헌에 이 말이 나타나기 시작한 것...,mrc-0-003956,드래곤의 암수 구분이 있는 신화는?


In [60]:
qgqg = f.loc[iiii3]
ii = list(bba['id'])
cc = list(bba['context'])
qqu = list(bba['question'])
aan = list(bba['answers'])

aas.extend(aan)
ccs.extend(cc)
iis.extend(ii)
qqs.extend(qqu)

qgqg4 = pd.DataFrame(data={'answers': aas, 'context': ccs, 'id': iis, 'question': qqs})
qgqg4

,answers,context,id,question
0,"{'answer_start': [235], 'text': ['하원']}",미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,mrc-1-000067,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?
1,"{'answer_start': [212], 'text': ['《경영의 실제》']}",'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,mrc-0-004397,현대적 인사조직관리의 시발점이 된 책은?
2,"{'answer_start': [510], 'text': ['백성']}",강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,mrc-1-000362,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?
3,"{'answer_start': [625], 'text': ['중국']}","불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",mrc-0-001510,11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?
4,"{'answer_start': [30], 'text': ['4개']}",동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,mrc-0-000823,명문이 적힌 유물을 구성하는 그릇의 총 개수는?
...,...,...,...,...
5567,"{'text': ['국민'], 'answer_start': [162]}",1947년 6월 미군정 사령관 존 하지는 이승만에게 이승만과 김구가 계획 중이라는 ...,mrc-qg-23783,여운형이 최우선으로 생각했던 것은?
5568,"{'text': ['남쪽'], 'answer_start': [83]}",전날 밤부터 난지도쪽으로 한강을 도하한 1개 연대 규모의 북한군이 오류동 정면으로 ...,mrc-qg-23784,북한군이 공격에 앞서 비행장쪽에서 그들의 보병에 화력을 지원하던 전차 2대를 야음을...
5569,"{'text': ['소련'], 'answer_start': [64]}",1969년에 연정에 합의한 사민당과 자유민주당의 사회자유 연정은 기존 외교 정책에서...,mrc-qg-23785,1969년 사민당과 자유민주당의 사회자유 연정이 우호적 입장을 취한 나라는 동독과 ...
5570,"{'text': ['미국'], 'answer_start': [37]}",오이 샌드위치는 영국이 기원인 샌드위치의 하나이다. 현대의 다른 종류의 오이샌드위치...,mrc-qg-23786,현대의 오이샌드위치는 대부분 어느 나라에서 만들어지나요?


In [565]:
!rm -rf qg_dataset_top_4

In [61]:
Dataset.from_pandas(qgqg4).save_to_disk(f"qg_filters")

In [62]:
!tar -cvf qg_filters.tar qg_filters

qg_filters/
qg_filters/state.json
qg_filters/dataset_info.json
qg_filters/dataset.arrow


In [63]:
hhhhhh = load_from_disk('qg_filters')
hhhhhh

Dataset({
    features: ['answers', 'context', 'id', 'question'],
    num_rows: 5572
})